# 透過機器學習預測股市漲跌-基本資料處理
## 作者：蔡尚宏(臺灣行銷研究特邀作者)、劉睿哲(臺灣行銷研究特邀作者)、鄭晴文(臺灣行銷研究特邀作者)
## 原始資料請見[本連結](https://www.kaggle.com/aaron7sun/stocknews)，下載下來後將pd.read_csv()內的路徑改為資料集的路徑即可

In [1]:
import os
import re
import nltk
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datetime import timedelta, datetime
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vivia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vivia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data = pd.read_csv(r'C:\Users\vivia\Downloads\archive\Combined_News_DJIA.csv')

In [4]:
def preprocess(processdata):
    # 將所有字母轉為小寫
    headlines = []
    for i in range(1, 21):
      headlines.append('Top'+str(i))
    processdata[headlines] = processdata[headlines].astype(str)
    processdata[headlines] = processdata[headlines].applymap(str.lower)
    
    # 組成以天為單位的data
    processdata_headlines = []
    for row in range(0,len(processdata.index)):
      processdata_headlines.append(' '.join(str(x) for x in processdata.iloc[row,2:27]))

    # 刪除數字與標點符號
    for line in range(len(processdata_headlines)):
      processdata_headlines[line] = re.sub(r'[^A-Za-z]'," ", processdata_headlines[line])

    # 將標題切為個別字詞
    for sentence in range(len(processdata_headlines)):
      processdata_headlines[sentence] = word_tokenize(processdata_headlines[sentence]) 

    # 去除英文字母及停用詞
    alpha = []
    for abc in string.ascii_lowercase :
      alpha.append(abc)      
    en_stops = stopwords.words('english')
    en_stops.extend(alpha)
    for sentence in range(len(processdata_headlines)):
      processdata_headlines[sentence] = [w for w in processdata_headlines[sentence] if w not in en_stops] 
    
    # 詞幹提取與詞形還原
    for sentence in range(len(processdata_headlines)):
      processdata_headlines[sentence] = [WordNetLemmatizer().lemmatize(w) for w in processdata_headlines[sentence]]
      processdata_headlines[sentence] = [WordNetLemmatizer().lemmatize(w, pos='v') for w in processdata_headlines[sentence]]   

    # 組回標題
    final_processdata_headlines = []
    for words in processdata_headlines :
      filter_words = ""
      for i in range(len(words)) :
        filter_words = filter_words + words[i] + " "
      final_processdata_headlines.append(filter_words)  

    return final_processdata_headlines

### 資料按照時間切

In [5]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']
final_traindata = preprocess(train)
final_testdata = preprocess(test)

C:\Users\vivia\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### 資料處理完的結果

In [6]:
# 此為第一天25則標題處理完，並串聯而成的句子
print(final_traindata[0])

georgia down two russian warplane country move brink war break musharraf impeach russia today column troop roll south ossetia footage fight youtube russian tank move towards capital south ossetia reportedly completely destroy georgian artillery fire afghan child rap impunity official say sick three year old rap nothing russian tank enter south ossetia whilst georgia shoot two russian jet break georgia invade south ossetia russia warn would intervene side enemy combatent trial nothing sham salim haman sentence year keep longer anyway feel like georgian troop retreat osettain capital presumably leave several hundred people kill video prep georgia war russia rice give green light israel attack iran say veto israeli military ops announce class action lawsuit behalf american public fbi russia georgia war nyt top story open ceremony olympics fuck disgrace yet proof decline journalism china tell bush stay country affair world war iii start today georgia invade south ossetia russia get involve